In [21]:
from util.utils import *
import os

In [22]:
## load datasets
def loadDataframes():
    cat62 = openDataframe("cat62_bronze.csv")
    espera = openDataframe("espera_bronze.csv")
    bimtra = openDataframe("bimtra_bronze.csv")
    metaf = openDataframe("metaf_bronze.csv")
    metar = openDataframe("metar_bronze.csv")
    tc_prev = openDataframe("tc-prev_bronze.csv")
    tc_real = openDataframe("tc-real_bronze.csv")
    return cat62, espera, bimtra, metaf, metar, tc_prev, tc_real

cat62, espera, bimtra, metaf, metar, tc_prev, tc_real = loadDataframes()

In [23]:
## column names of each dataframe
def getColumns():
    cat62_cols = cat62.columns
    espera_cols = espera.columns
    bimtra_cols = bimtra.columns
    metaf_cols = metaf.columns
    metar_cols = metar.columns
    tc_prev_cols = tc_prev.columns
    tc_real_cols = tc_real.columns
    return cat62_cols, espera_cols, bimtra_cols, metaf_cols, metar_cols, tc_prev_cols, tc_real_cols

cat62_cols, espera_cols, bimtra_cols, metaf_cols, metar_cols, tc_prev_cols, tc_real_cols = getColumns()
print(cat62_cols)
print(espera_cols)
print(bimtra_cols)
print(metaf_cols)
print(metar_cols)
print(tc_prev_cols)
print(tc_real_cols)

Index(['flightid', 'lat', 'lon', 'flightlevel', 'speed', 'dt_radar',
       'dt_radar_formatted'],
      dtype='object')
Index(['esperas', 'hora', 'aero', 'hora_formatted'], dtype='object')
Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'dt_dep_formatted',
       'dt_arr_formatted'],
      dtype='object')
Index(['hora', 'metaf', 'aero', 'hora_formatted'], dtype='object')
Index(['hora', 'metar', 'aero', 'hora_formatted'], dtype='object')
Index(['hora', 'troca', 'aero', 'hora_formatted'], dtype='object')
Index(['hora', 'nova_cabeceira', 'antiga_cabeceira', 'aero', 'hora_formatted'], dtype='object')


In [24]:
## merge cat62 and bimtra dataframes with flightid
def mergeWithFlightID(cat62, bimtra):
    cat62_bimtra = pd.merge(cat62, bimtra, on="flightid")
    return cat62_bimtra

merged_df = mergeWithFlightID(cat62, bimtra)
merged_df.head()

,flightid,lat,lon,flightlevel,speed,dt_radar,dt_radar_formatted,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted
0,ce43268d98c9e81575d0efeb92383aba,-0.360540,-0.768632,260.0,341.0,1677629941000,"Tuesday, February 28, 2023 21:19:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
1,ce43268d98c9e81575d0efeb92383aba,-0.370018,-0.770168,260.0,333.0,1677630303000,"Tuesday, February 28, 2023 21:25:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
2,ce43268d98c9e81575d0efeb92383aba,-0.377761,-0.771272,260.0,349.0,1677630601000,"Tuesday, February 28, 2023 21:30:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
3,ce43268d98c9e81575d0efeb92383aba,-0.393593,-0.769379,92.0,291.0,1677631744000,"Tuesday, February 28, 2023 21:49:04",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"
4,ce43268d98c9e81575d0efeb92383aba,-0.400201,-0.768154,90.0,255.0,1677632283000,"Tuesday, February 28, 2023 21:58:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56"


In [25]:
## crie uma função que pega a coluna data_dep e "arredonda" para a hora atual inteira
## exemplo: se data_dep = 2019-01-01 12:34:56, a função deve retornar 2019-01-01 12:00:00
## a hora em dt_dep esta no seguinte formato: 1677629154000, 1677629154000, 1677629154000, etc
def roundHour(dt_dep):
    dt_dep = dt_dep/1000
    dt_dep = datetime.fromtimestamp(dt_dep)
    dt_dep = dt_dep.replace(minute=0, second=0, microsecond=0)
    return dt_dep


merged_df['hora_ref'] = merged_df['dt_dep'].apply(roundHour)
merged_df.head()

,flightid,lat,lon,flightlevel,speed,dt_radar,dt_radar_formatted,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_ref
0,ce43268d98c9e81575d0efeb92383aba,-0.360540,-0.768632,260.0,341.0,1677629941000,"Tuesday, February 28, 2023 21:19:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
1,ce43268d98c9e81575d0efeb92383aba,-0.370018,-0.770168,260.0,333.0,1677630303000,"Tuesday, February 28, 2023 21:25:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
2,ce43268d98c9e81575d0efeb92383aba,-0.377761,-0.771272,260.0,349.0,1677630601000,"Tuesday, February 28, 2023 21:30:01",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
3,ce43268d98c9e81575d0efeb92383aba,-0.393593,-0.769379,92.0,291.0,1677631744000,"Tuesday, February 28, 2023 21:49:04",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00
4,ce43268d98c9e81575d0efeb92383aba,-0.400201,-0.768154,90.0,255.0,1677632283000,"Tuesday, February 28, 2023 21:58:03",SBCF,SBRJ,1677629154000,1677633476000,"Tuesday, February 28, 2023 21:05:54","Tuesday, February 28, 2023 22:17:56",2023-02-28 21:00:00


In [26]:
def groupById(df):
    grouped = df.groupby(['flightid', 'hora_ref'])  # Agrupe por flightid e hora_ref
    return grouped

def createCoordinatesList(group):
    lat_lon_list = [(row['lat'], row['lon']) for _, row in group.iterrows()]
    return lat_lon_list

def formatDataframe(df):
    grouped = groupById(df)
    
    dados_final = {
        'flightid': [],
        'origem': [],
        'destino': [],
        'dt_dep': [],
        'hora_ref': [],
        'snapshot_radar': [],
    }
    
    for (flight_id, hora_ref), group in grouped:  # Altere aqui para iterar corretamente sobre os grupos
        lat_lon_list = createCoordinatesList(group)
        
        origem = group['origem'].iloc[0]
        destino = group['destino'].iloc[0]
        dt_dep = group['dt_dep'].iloc[0]
        
        dados_final['flightid'].append(flight_id)
        dados_final['origem'].append(origem)
        dados_final['destino'].append(destino)
        dados_final['dt_dep'].append(dt_dep)
        dados_final['hora_ref'].append(hora_ref)
        dados_final['snapshot_radar'].append(lat_lon_list)
    
    new_df = pd.DataFrame(dados_final)
    return new_df

formated_df = formatDataframe(merged_df)
formated_df.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar
0,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116..."
1,0001d7d56ab8aaa1fd7e8fb4ae079f60,SBBR,SBCT,1677700968000,2023-03-01 17:00:00,"[(-0.345388457, -0.8614051214), (-0.3590661267..."
2,0004796cee4930d622b6eabdc22cdb5b,SBPA,SBBR,1677832349000,2023-03-03 05:00:00,"[(-0.4760178852, -0.8794155068), (-0.478144077..."
3,000897ae4e502951775f08651ea4d663,SBSP,SBBR,1680137219000,2023-03-29 21:00:00,"[(-0.4128358742, -0.817906891), (-0.4127446866..."
4,000988161c4bb5bd05e0dfba694085b4,SBPA,SBFL,1679014304000,2023-03-16 21:00:00,"[(-0.5234733354, -0.8926822638), (-0.521382814..."


In [27]:
len(formated_df)

26792

In [28]:
espera.head()

,esperas,hora,aero,hora_formatted
0,0,1677628800000,SBBR,"Tuesday, February 28, 2023 21:00:00"
1,0,1677632400000,SBBR,"Tuesday, February 28, 2023 22:00:00"
2,0,1677636000000,SBBR,"Tuesday, February 28, 2023 23:00:00"
3,0,1677639600000,SBBR,"Wednesday, March 01, 2023 00:00:00"
4,0,1677643200000,SBBR,"Wednesday, March 01, 2023 01:00:00"


In [29]:
formated_df.columns

Index(['flightid', 'origem', 'destino', 'dt_dep', 'hora_ref',
       'snapshot_radar'],
      dtype='object')

In [30]:
## merge espera with hora_ref
def mergeByRefDate(df, espera):
    merged_df = pd.merge(df, espera, on=['hora_ref'])
    ## drop 'hora' column
    merged_df = merged_df.drop(columns=['hora'])
    return merged_df

espera['hora_ref'] = espera['hora'].apply(roundHour)
formated_df = mergeByRefDate(formated_df, espera)
formated_df.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,esperas,aero,hora_formatted
0,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBBR,"Saturday, March 18, 2023 08:00:00"
1,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBCF,"Saturday, March 18, 2023 08:00:00"
2,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBCT,"Saturday, March 18, 2023 08:00:00"
3,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBFL,"Saturday, March 18, 2023 08:00:00"
4,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBGL,"Saturday, March 18, 2023 08:00:00"


In [31]:
formated_df.hora_ref.unique()

array(['2023-03-18T08:00:00.000000000', '2023-03-01T17:00:00.000000000',
       '2023-03-03T05:00:00.000000000', '2023-03-29T21:00:00.000000000',
       '2023-03-16T21:00:00.000000000', '2023-03-11T02:00:00.000000000',
       '2023-03-03T11:00:00.000000000', '2023-03-30T06:00:00.000000000',
       '2023-03-02T08:00:00.000000000', '2023-03-26T11:00:00.000000000',
       '2023-03-30T15:00:00.000000000', '2023-03-09T11:00:00.000000000',
       '2023-03-21T12:00:00.000000000', '2023-03-28T10:00:00.000000000',
       '2023-03-27T08:00:00.000000000', '2023-03-30T18:00:00.000000000',
       '2023-03-10T06:00:00.000000000', '2023-03-12T21:00:00.000000000',
       '2023-03-20T21:00:00.000000000', '2023-03-05T22:00:00.000000000',
       '2023-03-20T11:00:00.000000000', '2023-03-22T23:00:00.000000000',
       '2023-03-12T14:00:00.000000000', '2023-03-09T14:00:00.000000000',
       '2023-03-09T17:00:00.000000000', '2023-03-12T17:00:00.000000000',
       '2023-03-08T15:00:00.000000000', '2023-03-01

In [32]:
def convertHour(hora_ref):
    # Formate a hora_ref para corresponder ao formato no nome do arquivo
    hora_formatada_str = hora_ref.strftime('%Y%m%d%H%M')
    hora_formatada = datetime.strptime(hora_formatada_str, '%Y%m%d%H%M')
    return hora_formatada


def findImageByHour(imagesDir, refHour):
    # Converter a hora_ref para o formato desejado
    formatedDate = refHour.strftime('%Y%m%d')
    convertedHour = convertHour(refHour)
    
    # Construir o nome do arquivo com base na data_formatada
    fileName = f'S11635384_{convertedHour}.jpg'
    
    # Construir o caminho completo do arquivo
    filePath = os.path.join(imagesDir, fileName)
    
    # Verificar se o arquivo existe
    if os.path.isfile(filePath):
        return filePath
    else:
        return None  # Arquivo não encontrado

imagesDir = '../images/'
formated_df['path'] = formated_df['hora_ref'].apply(lambda x: findImageByHour(imagesDir, x))
formated_df.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,esperas,aero,hora_formatted,path
0,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBBR,"Saturday, March 18, 2023 08:00:00",None
1,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBCF,"Saturday, March 18, 2023 08:00:00",None
2,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBCT,"Saturday, March 18, 2023 08:00:00",None
3,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBFL,"Saturday, March 18, 2023 08:00:00",None
4,00010ed6d09c78b5a37f4df13f524f56,SBSV,SBSP,1679137732000,2023-03-18 08:00:00,"[(-0.2288001081, -0.668855167), (-0.2414646116...",0,SBGL,"Saturday, March 18, 2023 08:00:00",None


In [33]:
formated_df.path.unique()

array([None], dtype=object)

In [34]:
formated_df.to_csv('formated_df.csv', index=False)